# Financial Data Extractor Tool

A robust tool designed to understand quarterly financial reports and extract key financial metrics using OpenAI API.

**Target Metrics:**
- Total Revenue
- Net Profit
- Operating Margin
- Additional financial KPIs

## 1. Environment Setup

In [1]:
# Import required libraries
import os
import pandas as pd
import json
from dotenv import load_dotenv
from typing import Dict, List, Optional

In [2]:
# PDF processing libraries
import PyPDF2
import pdfplumber
import fitz  # pymupdf

In [4]:
# OpenAI setup
from openai import OpenAI

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPEN_API_KEY'))
print("✅ OpenAI client initialized")

✅ OpenAI client initialized


## 2. PDF Reading Functions

In [5]:
def extract_text_pypdf2(pdf_path: str) -> str:
    """Extract text using PyPDF2."""
    text = ""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        print(f"PyPDF2 extraction failed: {e}")
    return text

In [6]:
def extract_text_pdfplumber(pdf_path: str) -> str:
    """Extract text using pdfplumber."""
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
    except Exception as e:
        print(f"pdfplumber extraction failed: {e}")
    return text

In [7]:
def extract_text_pymupdf(pdf_path: str) -> str:
    """Extract text using PyMuPDF."""
    text = ""
    try:
        doc = fitz.open(pdf_path)
        for page in doc:
            text += page.get_text() + "\n"
        doc.close()
    except Exception as e:
        print(f"PyMuPDF extraction failed: {e}")
    return text

In [8]:
def extract_pdf_text(pdf_path: str) -> str:
    """Extract text using multiple methods for robustness."""
    print(f"Extracting text from: {os.path.basename(pdf_path)}")
    
    # Try pdfplumber first (usually best for tables)
    text = extract_text_pdfplumber(pdf_path)
    if len(text.strip()) > 100:
        print("✅ pdfplumber extraction successful")
        return text
    
    # Fallback to PyMuPDF
    text = extract_text_pymupdf(pdf_path)
    if len(text.strip()) > 100:
        print("✅ PyMuPDF extraction successful")
        return text
    
    # Last resort: PyPDF2
    text = extract_text_pypdf2(pdf_path)
    if len(text.strip()) > 100:
        print("✅ PyPDF2 extraction successful")
        return text
    
    raise Exception("All PDF extraction methods failed")

## 3. OpenAI Configuration

In [53]:
FINANCIAL_EXTRACTION_PROMPT = """
You are a financial data extraction expert. Your task is to extract **detailed structured information** from quarterly and annual financial reports.  


### Extraction Rules:
1. **Detailed Sections** (capture entire tables or narrative if present):
   - **Performance Highlights**: revenue, profit, demand, people, cash flow.  
   - **Growth Summary**: year-over-year growth, constant currency growth, margins.  
   - **Cash Flow Summary**: cash from operations, free cash flow, dividends, buyback, ratios.  
   - **Balance Sheet**: assets, liabilities, equity breakdown.  
   - **Growth by Market**: geographical revenue shares and CC growth.  
   - **Growth by Domain**: vertical/industry revenue shares and CC growth.  
   - **Client Metrics**: number of clients in revenue bands ($1m+, $10m+, $100m+, etc.).  
   - **Human Resources**: headcount, attrition %, diversity, learning hours, competencies.   
   - **Notes**: Just dont extract numbers but also Context which can be later compared with all quaters and can be used to compare and predict for future. 



### Output Format
Return ONLY a valid JSON object:

```json
{
  "period": ,
  "metrics": {
    "total_revenue": ,
    "net_profit": ,
    "operating_income": ,
    "operating_margin_percent": ,
    "net_margin_percent": ,
    "ebitda": null,
    "gross_profit": ,
    "operating_expenses": ,
    "tax_expense": ,
    "basic_eps": ,
    "total_assets": ,
    "total_liabilities": ,
    "shareholders_equity": ,
  },
  "sections": {
    "performance_highlights": { ... },
    "growth_summary": { ... },
    "cash_flow_summary": { ... },
    "balance_sheet": { ... },
    "growth_by_market": { ... },
    "growth_by_domain": { ... },
  }
}
"""

## 4. FinancialDataExtractorTool Class

In [54]:
class FinancialDataExtractorTool:
    """A robust tool for extracting financial metrics from quarterly reports."""
    
    def __init__(self, openai_client):
        self.client = openai_client
        self.model = "gpt-4-turbo"
    
    def extract_financial_data(self, pdf_path: str) -> Dict:
        """Extract financial metrics from a PDF report."""
        try:
            # Extract text from PDF
            text = extract_pdf_text(pdf_path)
            
            # Truncate text if too long (OpenAI token limits)
            if len(text) > 50000:
                text = text[:50000] + "\n[Document truncated due to length]"
            
            # Call OpenAI API
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": FINANCIAL_EXTRACTION_PROMPT},
                    {"role": "user", "content": f"Extract financial data from this quarterly report:\n\n{text}"}
                ],
                temperature=0.1,
                max_tokens=2500
            )
            
            # Parse JSON response
            result_text = response.choices[0].message.content.strip()
            
            # Clean JSON response (remove markdown code blocks if present)
            if result_text.startswith('```json'):
                result_text = result_text[7:-3]
            elif result_text.startswith('```'):
                result_text = result_text[3:-3]
            
            financial_data = json.loads(result_text)
            
            # Add source file information
            financial_data['source_file'] = os.path.basename(pdf_path)
            
            return financial_data
            
        except json.JSONDecodeError as e:
            print(f"JSON parsing error: {e}")
            print(f"Raw response: {result_text}")
            return None
        except Exception as e:
            print(f"Extraction error: {e}")
            return None

In [55]:
# Initialize the extractor tool
extractor = FinancialDataExtractorTool(client)
print("✅ FinancialDataExtractorTool initialized")

✅ FinancialDataExtractorTool initialized


## 5. Define PDF Files

In [56]:
# Define paths to quarterly PDF files
pdf_files = [
    "../data/pdfs/Quarter I Ended FY 2025-26.pdf",
    "../data/pdfs/Quarter III Ended FY 2024-25.pdf", 
    "../data/pdfs/Quarter IV & Year Ended FY 2024-25.pdf"
]

# Verify files exist
for pdf_file in pdf_files:
    if os.path.exists(pdf_file):
        print(f"✅ Found: {os.path.basename(pdf_file)}")
    else:
        print(f"❌ Missing: {pdf_file}")

✅ Found: Quarter I Ended FY 2025-26.pdf
✅ Found: Quarter III Ended FY 2024-25.pdf
✅ Found: Quarter IV & Year Ended FY 2024-25.pdf


## 6. Single PDF Test

In [57]:
# Test extraction on first PDF
test_pdf = pdf_files[0]
print(f"Testing extraction on: {os.path.basename(test_pdf)}")
print("-" * 50)

test_result = extractor.extract_financial_data(test_pdf)

if test_result:
    print("✅ Extraction successful!")
    print(json.dumps(test_result, indent=2))
else:
    print("❌ Extraction failed")

Testing extraction on: Quarter I Ended FY 2025-26.pdf
--------------------------------------------------
Extracting text from: Quarter I Ended FY 2025-26.pdf


Cannot set gray non-stroke color because /'P47' is an invalid float value
Cannot set gray non-stroke color because /'P53' is an invalid float value
Cannot set gray non-stroke color because /'P59' is an invalid float value
Cannot set gray non-stroke color because /'P65' is an invalid float value
Cannot set gray non-stroke color because /'P71' is an invalid float value
Cannot set gray non-stroke color because /'P86' is an invalid float value
Cannot set gray non-stroke color because /'P92' is an invalid float value
Cannot set gray non-stroke color because /'P98' is an invalid float value
Cannot set gray non-stroke color because /'P104' is an invalid float value
Cannot set gray non-stroke color because /'P110' is an invalid float value
Cannot set gray non-stroke color because /'P119' is an invalid float value
Cannot set gray non-stroke color because /'P125' is an invalid float value
Cannot set gray non-stroke color because /'P131' is an invalid float value
Cannot set gray non-stroke color 

✅ pdfplumber extraction successful
✅ Extraction successful!
{
  "period": "Q1 FY 2025-26",
  "metrics": {
    "total_revenue": 634370,
    "net_profit": 122240,
    "operating_income": null,
    "operating_margin_percent": 24.5,
    "net_margin_percent": 20.1,
    "ebitda": null,
    "gross_profit": null,
    "operating_expenses": null,
    "tax_expense": null,
    "basic_eps": null,
    "total_assets": 1685860,
    "total_liabilities": 675300,
    "shareholders_equity": 1000360
  },
  "sections": {
    "performance_highlights": {
      "revenue": {
        "INR": 634370,
        "USD": 7421,
        "constant_currency": -3.1
      },
      "profit": {
        "operating_margin": 24.5,
        "net_margin": 20.1,
        "cash_flow_operations_percent_net_profit": 100.3
      },
      "demand": {
        "clients_100M_plus": -1,
        "clients_10M_plus": 9,
        "clients_1M_plus": 26,
        "order_book_TCV": 9.4,
        "geographical_TCV": {
          "North_America": 4.4,
     

## 7. Batch Processing All Quarterly PDFs

In [58]:
# Process all quarterly PDFs
results = []

for pdf_file in pdf_files:
    print(f"\nProcessing: {os.path.basename(pdf_file)}")
    print("=" * 60)
    
    result = extractor.extract_financial_data(pdf_file)
    
    if result:
        results.append(result)
        print(f"✅ Successfully extracted data from {result['source_file']}")
        print(f"Quarter: {result.get('quarter', 'N/A')}")
        print(f"Revenue: {result['metrics'].get('total_revenue', 'N/A')}")
        print(f"Net Profit: {result['metrics'].get('net_profit', 'N/A')}")
    else:
        print(f"❌ Failed to extract data from {os.path.basename(pdf_file)}")

print(f"\n🎯 Successfully processed {len(results)} out of {len(pdf_files)} files")


Processing: Quarter I Ended FY 2025-26.pdf
Extracting text from: Quarter I Ended FY 2025-26.pdf


Cannot set gray non-stroke color because /'P47' is an invalid float value
Cannot set gray non-stroke color because /'P53' is an invalid float value
Cannot set gray non-stroke color because /'P59' is an invalid float value
Cannot set gray non-stroke color because /'P65' is an invalid float value
Cannot set gray non-stroke color because /'P71' is an invalid float value
Cannot set gray non-stroke color because /'P86' is an invalid float value
Cannot set gray non-stroke color because /'P92' is an invalid float value
Cannot set gray non-stroke color because /'P98' is an invalid float value
Cannot set gray non-stroke color because /'P104' is an invalid float value
Cannot set gray non-stroke color because /'P110' is an invalid float value
Cannot set gray non-stroke color because /'P119' is an invalid float value
Cannot set gray non-stroke color because /'P125' is an invalid float value
Cannot set gray non-stroke color because /'P131' is an invalid float value
Cannot set gray non-stroke color 

✅ pdfplumber extraction successful
✅ Successfully extracted data from Quarter I Ended FY 2025-26.pdf
Quarter: N/A
Revenue: 634370
Net Profit: 122240

Processing: Quarter III Ended FY 2024-25.pdf
Extracting text from: Quarter III Ended FY 2024-25.pdf
✅ pdfplumber extraction successful
✅ Successfully extracted data from Quarter III Ended FY 2024-25.pdf
Quarter: N/A
Revenue: 639730
Net Profit: 123800

Processing: Quarter IV & Year Ended FY 2024-25.pdf
Extracting text from: Quarter IV & Year Ended FY 2024-25.pdf


Cannot set gray non-stroke color because /'P51' is an invalid float value
Cannot set gray non-stroke color because /'P57' is an invalid float value
Cannot set gray non-stroke color because /'P63' is an invalid float value
Cannot set gray non-stroke color because /'P69' is an invalid float value
Cannot set gray non-stroke color because /'P75' is an invalid float value
Cannot set gray non-stroke color because /'P90' is an invalid float value
Cannot set gray non-stroke color because /'P96' is an invalid float value
Cannot set gray non-stroke color because /'P102' is an invalid float value
Cannot set gray non-stroke color because /'P108' is an invalid float value
Cannot set gray non-stroke color because /'P114' is an invalid float value
Cannot set gray non-stroke color because /'P123' is an invalid float value
Cannot set gray non-stroke color because /'P129' is an invalid float value
Cannot set gray non-stroke color because /'P135' is an invalid float value
Cannot set gray non-stroke color

✅ pdfplumber extraction successful
✅ Successfully extracted data from Quarter IV & Year Ended FY 2024-25.pdf
Quarter: N/A
Revenue: 644790
Net Profit: 122240

🎯 Successfully processed 3 out of 3 files


## 8. Results Analysis and Visualization

In [59]:
# Create a comprehensive results DataFrame
if results:
    # Flatten results for DataFrame
    flattened_data = []
    
    for result in results:
        row = {
            'source_file': result['source_file'],
            'quarter': result.get('quarter', 'N/A'),
            'currency': result.get('currency', 'N/A')
        }
        
        # Add all metrics
        metrics = result.get('metrics', {})
        for key, value in metrics.items():
            row[key] = value
        
        flattened_data.append(row)
    
    # Create DataFrame
    df = pd.DataFrame(flattened_data)
    
    # Display results
    print("📊 Financial Metrics Summary")
    print("=" * 50)
    display(df)
    
else:
    print("❌ No successful extractions to analyze")

📊 Financial Metrics Summary


,source_file,quarter,currency,total_revenue,net_profit,operating_income,operating_margin_percent,net_margin_percent,ebitda,gross_profit,operating_expenses,tax_expense,basic_eps,total_assets,total_liabilities,shareholders_equity
0,Quarter I Ended FY 2025-26.pdf,N/A,N/A,634370,122240,NaN,24.5,20.10,None,NaN,NaN,NaN,NaN,1685860,675300,1000360
1,Quarter III Ended FY 2024-25.pdf,N/A,N/A,639730,123800,156570.0,24.5,19.35,None,248340.0,91770.0,42220.0,34.21,1743110,641330,1094880
2,Quarter IV & Year Ended FY 2024-25.pdf,N/A,N/A,644790,122240,156010.0,24.2,19.00,None,250050.0,94040.0,41090.0,33.79,1617550,638580,968820


In [60]:
# Key metrics comparison
if results and len(results) > 1:
    key_metrics = ['total_revenue', 'net_profit', 'operating_margin', 'ebitda']
    
    comparison_data = []
    for result in results:
        row = {
            'Quarter': result.get('quarter', result['source_file']),
        }
        
        for metric in key_metrics:
            value = result['metrics'].get(metric)
            row[metric.replace('_', ' ').title()] = value if value is not None else 'N/A'
        
        comparison_data.append(row)
    
    comparison_df = pd.DataFrame(comparison_data)
    
    print("\n📈 Key Metrics Comparison")
    print("=" * 40)
    display(comparison_df)


📈 Key Metrics Comparison


,Quarter,Total Revenue,Net Profit,Operating Margin,Ebitda
0,Quarter I Ended FY 2025-26.pdf,634370,122240,N/A,N/A
1,Quarter III Ended FY 2024-25.pdf,639730,123800,N/A,N/A
2,Quarter IV & Year Ended FY 2024-25.pdf,644790,122240,N/A,N/A


## 9. Export Results

In [61]:
# Export to JSON
if results:
    output_dir = "../outputs"
    os.makedirs(output_dir, exist_ok=True)
    
    # Save detailed JSON
    json_output_path = os.path.join(output_dir, "financial_extraction_results.json")
    with open(json_output_path, 'w') as f:
        json.dump(results, f, indent=2)
    print(f"✅ Detailed results saved to: {json_output_path}")
    
    # Save CSV summary
    csv_output_path = os.path.join(output_dir, "financial_metrics_summary.csv")
    df.to_csv(csv_output_path, index=False)
    print(f"✅ CSV summary saved to: {csv_output_path}")
    
    print(f"\n🎯 Pipeline completed successfully!")
    print(f"📊 Extracted data from {len(results)} quarterly reports")
    
else:
    print("❌ No results to export")

✅ Detailed results saved to: ../outputs/financial_extraction_results.json
✅ CSV summary saved to: ../outputs/financial_metrics_summary.csv

🎯 Pipeline completed successfully!
📊 Extracted data from 3 quarterly reports
